# Código para Unir los datasets

## Instalar Librerías

In [ ]:
#instala pandas
%pip install pandas

## Primer modelo
Cantidad de datos: 166475

In [5]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import datetime as datetime

# Enlaces de tus archivos
url_discapacidad = 'https://drive.google.com/file/d/1k5tO-1iZ5FGWMJuykYylvbyafiXIMnZ-/view?usp=drive_link'
url_educacion = 'https://drive.google.com/file/d/1RBDPcDPG2mCV6dQbUlEJYy8KpnuDPAJH/view?usp=drive_link'
url_postulante = 'https://drive.google.com/file/d/1SafIdXKzpLRDJrCQuMAyJh-vE84Zv-P8/view?usp=drive_link'

# Función para convertir el enlace de Google Drive en enlace directo
def get_drive_link(url):
    file_id = url.split('/')[-2]
    return f'https://drive.google.com/uc?id={file_id}'

# 1. Leer los archivos
try:
    # Leer postulante
    df_postulante = pd.read_csv(get_drive_link(url_postulante))
    print("Archivo de postulante leído correctamente")

    # Leer educación
    df_educacion = pd.read_csv(get_drive_link(url_educacion),
                              sep=';',
                              quotechar='"',
                              encoding='utf-8',
                              engine='c',
                              on_bad_lines='skip')
    print("Archivo de educación leído correctamente")

    # Leer discapacidad
    df_discapacidad = pd.read_csv(get_drive_link(url_discapacidad))
    print("Archivo de discapacidad leído correctamente")

    #Leer DATA_EXPERIENCIASLABORALES
    df = pd.read_csv('DATA_EXPERIENCIASLABORALES.csv', sep=';')

    #imprimir 5 registros de DATA_EXPERIENCIASLABORALES
    print(df.head())

    # 2. Unir los DataFrames
    # Primero unimos postulante con educación
    df_merged = pd.merge(df_postulante,
                        df_educacion,
                        on='ID_POSTULANTE',
                        how='left')

    # Luego unimos con discapacidad (asegurándonos que las columnas coincidan)
    # Renombrar la columna en discapacidad si es necesario
    if 'DBIDPOSTULANTE' in df_discapacidad.columns:
        df_discapacidad = df_discapacidad.rename(columns={'DBIDPOSTULANTE': 'DOC_ID'})

    df_final = pd.merge(df_merged,
                       df_discapacidad,
                       on='DOC_ID',
                       how='left')
    
    #Unir con DATA_EXPERIENCIASLABORALES
    df_final = pd.merge(df_final,
                       df,
                       on='ID_POSTULANTE',
                       how='left')

    # 3. Limpieza básica
    # Rellenar valores nulos
    categorical_columns = ['GRADO', 'CARRERA', 'INSTITUCION', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']
    for col in categorical_columns:
        df_final[col] = df_final[col].fillna('No especificado')

    numeric_columns = ['ESTADO_CONADIS', 'DSCORE']
    for col in numeric_columns:
        df_final[col] = df_final[col].fillna(0)

    # 4. Guardar el resultado
    df_final.to_csv('datos_unidos_completos_59735.csv', index=False, encoding='utf-8')
    print("\nArchivo guardado exitosamente como 'datos_unidos_completos_59735.csv'")

    # 5. Mostrar resumen
    print("\nResumen del DataFrame final:")
    print(f"Número de filas: {len(df_final)}")
    print(f"Número de columnas: {len(df_final.columns)}")
    print("\nColumnas en el DataFrame final:", df_final.columns.tolist())
    print("\nValores no nulos en CAUSA y DSCORE:")
    print("CAUSA no nulos:", df_final['CAUSA'].notna().sum())
    print("DSCORE no nulos:", df_final['DSCORE'].notna().sum())

except Exception as e:
    print("Error en el proceso:", e)


Archivo de postulante leído correctamente
Archivo de educación leído correctamente
Archivo de discapacidad leído correctamente
                      ID_POSTULANTE                           EMPRESA  \
0  03AFDBD66E7929B125F8597834FA83A4  AAB3DD1AA8AF62CB516EE8C9AEEB0B35   
1  03AFDBD66E7929B125F8597834FA83A4  AAB3DD1AA8AF62CB516EE8C9AEEB0B35   
2  EA5D2F1C4608232E07D3AA3D998E5135  89C9A87CE323B6A3086D15356D3DB2BA   
3  EA5D2F1C4608232E07D3AA3D998E5135  89C9A87CE323B6A3086D15356D3DB2BA   
4  FC490CA45C00B1249BBE3554A4FDF6FB  8EAC8447385B013E2A83BAAAB6B47F35   

   FECHAINICIO    FECHAFIN                                        DESCRIPCION  \
0   20200301.0  20210701.0  capacitar a miembros de mesa, fuerzas armadas ...   
1   20200301.0  20210701.0  capacitar a miembros de mesa, fuerzas armadas ...   
2   20190601.0  20200901.0  ValorizaciÃ³n de los trabajos realizados mensu...   
3   20190601.0  20200901.0  ValorizaciÃ³n de los trabajos realizados mensu...   
4   20031001.0  20210101.0  j